In [3]:

import os
import shutil
import unittest

from arc.common import ARC_PATH, almost_equal_lists, read_yaml_file
from arc.job.adapters.pyscf import PYSCFAdapter
from arc.level import Level
from arc.species import ARCSpecies
from arc.species.vectors import calculate_distance, calculate_angle, calculate_dihedral_angle
from arc.utils.wip import work_in_progress



ModuleNotFoundError: No module named 'cython'

In [2]:

etoh_xyz = """ C       -0.93790674    0.28066443    0.10572942
                C        0.35659906   -0.44954997    0.05020174
                O        0.36626530   -1.59397979   -0.38012632
                H       -1.68923915   -0.33332195    0.61329151
                H       -0.85532021    1.23909997    0.62578027
                H       -1.30704889    0.46001151   -0.90948878
                H        0.76281007   -0.50036590    1.06483009
                H        1.04287051    0.12137561   -0.58236096
                H        1.27820018   -1.93031032   -0.35203473"""

In [3]:
        pyscf_buildinput = PYSCFAdapter(execution_type='incore',
                                    job_type='opt',
                                    project='test_2',
                                    project_directory=os.path.join(ARC_PATH, 'arc', 'testing', 'test_PYSCFAdapter'),
                                        level=Level(software='pyscf',
                                        method='b3lyp',
                                        basis='def2-TZVP',),
                                    species=[ARCSpecies(label='EtOH', smiles='CCO', xyz=etoh_xyz)],)


In [4]:
pyscf_buildinput.write_input_file()


In [5]:
import pandas as pd
# use pandas to read the yml file
df = pd.read_csv(os.path.join(ARC_PATH, 'arc', 'testing', 'test_PYSCFAdapter','calcs','Species','EtOH','opt_a10','input.yml'))

In [6]:
# read the yml file as a dictionary
input_dict = read_yaml_file(os.path.join(ARC_PATH, 'arc', 'testing', 'test_PYSCFAdapter','calcs','Species','EtOH','opt_a10','input.yml'))
input_dict

{'basis': 'def2-tzvp',
 'charge': 0,
 'is_ts': 'False',
 'job_type': 'opt',
 'restricted': 'False',
 'spin': 0,
 'xc_func': 'b3lyp',
 'xyz': 'C      -0.93790674    0.28066443    0.10572942\nC       0.35659906   -0.44954997    0.05020174\nO       0.36626530   -1.59397979   -0.38012632\nH      -1.68923915   -0.33332195    0.61329151\nH      -0.85532021    1.23909997    0.62578027\nH      -1.30704889    0.46001151   -0.90948878\nH       0.76281007   -0.50036590    1.06483009\nH       1.04287051    0.12137561   -0.58236096\nH       1.27820018   -1.93031032   -0.35203473'}

In [4]:
#!/usr/bin/env python3
# encoding: utf-8

"""
A standalone script to run PySCF with the xyz data outputted from ARC.
Should be run under the pyscf_env.
"""

# Parse the input.yml file

import os
import sys
import yaml
import argparse
import pyscf
from pyscf import gto, scf, dft

class PYSCFScript:
    
    def __init__(self, input_file):
        self.input_file = input_file
        self.input_dict = self.read_input_file()
        self.job_type = self.get_job_type()
        self.mol = self.get_mol()
        self.method = self.get_method()
        self.basis = self.get_basis()
        self.charge = self.get_charge()
        self.spin = self.get_spin()
        self.restricted = self.get_restricted()
        self.is_ts = self.get_ts_status()
        self.run()
    def read_input_file(self):
        # Check if input file is a YAML file
        if self.input_file.endswith('.yml'):
            with open(self.input_file, 'r') as f:
                input_dict = yaml.load(f, Loader=yaml.FullLoader)
        # Check if its a dict
        elif isinstance(self.input_file, dict):
            input_dict = self.input_file
        return input_dict
    def get_job_type(self):
        job_type = self.input_dict['job_type']
        return job_type
    def get_mol(self):
        mol = self.input_dict['xyz']
        return mol
    def get_method(self):
        method = self.input_dict['xc_func']
        return method
    def get_basis(self):
        basis = self.input_dict['basis']
        return basis
    def get_charge(self):
        charge = self.input_dict['charge']
        return charge
    def get_spin(self):
        spin = self.input_dict['spin']
        return spin
    def get_restricted(self):
        restricted = self.input_dict['restricted']
        return restricted
    def get_ts_status(self):
        is_ts = self.input_dict['is_ts']
        return is_ts
    def run(self):

        mol = gto.Mole()
        mol.atom = self.mol
        mol.basis = self.basis
        mol.charge = self.charge
        mol.spin = self.spin # TODO: Is this correct [multiplicity -> spin]?
        mol.build()
        
        if self.restricted:
            mf = dft.RKS(mol)
        else:
            mf = dft.UKS(mol)
        mf.xc = self.method
        
        # TODO: Check if TS
        if self.is_ts and self.job_type == 'opt':
            from pyscf.qsdopt.qsd_optimizer import QSD
            optimizer = QSD(mf, stationary_point='TS')
            

            # hess_update_freq: Frequency for numerical reevaluation of hessian. = 0 evaluates the numerical hessian in the first iteration and is updated with an BFGS rule, unless approaching a trap region, where it is reevaluated. (Default: 0)

            # numhess_method: Method for evaluating numerical hessian. Forward and central differences are available with “forward” and “central”, respectively. (Default: “forward”)

            # max_iter: Maximum number of optimization steps. (Default: 100)

            # step: Maximum norm between two optimization steps. (Default: 0.1)

            # hmin: Minimum distance between current geometry and stationary point of the quadratic form to consider convergence reached. (Default: 1e-6)

            # gthres: Gradient norm to consider convergence. (Default: 1e-5)

            optimizer.kernel(hess_update_freq=0, numhess_method='forward', max_iter=100, step=0.1, hmin=1e-6, gthres=1e-5)
        
        if self.job_type == 'opt' and not self.is_ts:
            from pyscf.geomopt.geometric_solver import optimize
            conv_params = { # These are the default settings
                                'convergence_energy': 1e-6,  # Eh
                                'convergence_grms': 3e-4,    # Eh/Bohr
                                'convergence_gmax': 4.5e-4,  # Eh/Bohr
                                'convergence_drms': 1.2e-3,  # Angstrom
                                'convergence_dmax': 1.8e-3,  # Angstrom
                            }
            mol_eq = optimize(mf, maxsteps=100, **conv_params) # TODO: Need to define maxsteps as a TRSH parameter
            mf.kernel()
            print('SCF energy of {0} is {1}.'.format(self.method, mf.e_tot))
            
        if self.job_type == 'freq':
            #npip install git+https://github.com/pyscf/properties
            from pyscf.prop.freq import rks            
            w, modes = rks.Freq(mf).kernel()            
            print('Frequencies (cm-1):')
        
        # if self.job_type == 'scan':
        #     #https://github.com/pyscf/pyscf/blob/14d88828cd1f18f1e5358da1445355bde55322a1/examples/scf/30-scan_pes.py#L16
        #     # Requires manual development for the code. Not feasible at the moment.
        #     scan_results = []
        #     for value in np.arange(self.start, self.end, self.step_size):
        #         upd
        if self.job_type == 'sp':
            mf.kernel()
            print('The single-point DFT energy is:', energy)
            
        print('SCF energy of {0} is {1}.'.format(self.method, mf.e_tot))
        
        

In [7]:
pyscf_class = PYSCFScript('/home/calvin/Code/ARC/arc/testing/test_PYSCFAdapter/calcs/Species/EtOH/opt_a205/input.yml')

converged SCF energy = -155.064984089535
--------------- RKS_Scanner gradients ---------------
         x                y                z
0 C     0.0245632818     0.0036414414     0.0080311187
1 C    -0.0346471223    -0.1790180869    -0.0751194040
2 O     0.0089993234     0.2174126780     0.0843159487
3 H     0.0008251925    -0.0021855254    -0.0014443389
4 H    -0.0028013905     0.0031935267     0.0015033104
5 H     0.0018732892    -0.0017309364    -0.0012752689
6 H    -0.0055553204    -0.0209332187    -0.0149920551
7 H    -0.0075226215    -0.0253092928    -0.0054298364
8 H     0.0142707736     0.0049474060     0.0044024496
----------------------------------------------
converged SCF energy = -155.064981630741
--------------- RKS_Scanner gradients ---------------
         x                y                z
0 C     0.0246144549     0.0036431146     0.0080316099
1 C    -0.0346690130    -0.1790161326    -0.0751208773
2 O     0.0089999323     0.2174147293     0.0843167243
3 H     0.000

In [20]:
# https://github.com/nmardirossian/PySCF_Tutorial/blob/master/user_guide.ipynb
import pyscf
from pyscf import gto, scf, dft
import numpy
import itertools
import pandas
import yaml
pandas.set_option('display.max_columns', 500)

class PYSCFScript_VB:
    
    def __init__(self, input_file) -> None:
        self.input_file = input_file
        self.input_dict = self.read_input_file()

        # Method
        self.method = self.get_method()
        # Molecule
        self.molecule = self.get_molecule()
        # Basis
        self.basis = self.get_basis()
        # Charge
        self.charge = self.get_charge()
        # Spin
        self.spin = self.get_spin()
        # Unit
        self.unit = self.get_unit()
        # Restricted
        self.restricted = self.get_restricted()
        # Conv tol
        self.conv_tol = self.get_conv_tol()
        # Conv Tol Grad
        self.conv_tol_grad = self.get_conv_tol_grad()
        # Direct scf tol
        self.direct_scf_tol = self.get_direct_scf_tol()
        # Init guess
        self.init_guess = self.get_init_guess()
        # Level Shift
        self.level_shift = self.get_level_shift()
        # Max cycles
        self.max_cycle = self.get_max_cycle()
        # Max memory
        self.max_memory = self.get_max_memory()
        # XC
        self.xc = self.get_xc()
        # NLC
        self.nlc = self.get_nlc()
        # xc grid
        self.xc_grid = self.get_xc_grid()
        # nlc grid
        self.nlc_grid = self.get_nlc_grid()
        # small_rho_cutoff
        self.small_rho_cutoff = self.get_small_rho_cutoff()
        # atomic radii
        self.atomic_radii = self.get_atomic_radii()
        # Becke scheme
        self.becke_scheme = self.get_becke_scheme()
        # Prune
        self.prune = self.get_prune()
        # Radi method
        self.radi_method = self.get_radi_method()
        # Radii Adjust
        self.radii_adjust = self.get_radii_adjust()
        # Algorithm
        self.algorithm = self.get_algorithm()
        # Datapath
        self.datapath = self.get_datapath()
        # getdict
        self.getdict = self.get_getdict()
        # lin depth
        self.lin_depth = self.get_lin_depth()
        # prop
        self.prop = self.get_prop()
        # scf type
        self.scf_type = self.get_scf_type()
        # Stable
        self.stable = self.get_stable()
        # Stable cyc
        self.stable_cyc = self.get_stable_cyc()
        # verbose
        self.verbose = self.get_verbose()
        # job type
        self.job_type = self.get_job_type()
        # is ts
        self.is_ts = self.get_ts_status()

        self.mol_eq = None
    def read_input_file(self):
        # Check if input file is a YAML file
        if isinstance(self.input_file, str) and self.input_file.endswith('.yml'):
            with open(self.input_file, 'r') as f:
                input_dict = yaml.load(f, Loader=yaml.FullLoader)
        elif isinstance(self.input_file, dict):
            input_dict = self.input_file
        return input_dict
    
    def get_method(self):
        # Get method in input dict if exists
        method = self.input_dict.get('xc_func', None)
        return method
    
    def get_molecule(self):
        # Get molecule in input dict if exists
        molecule = self.input_dict['xyz']
        return molecule
    
    def get_basis(self):
        # Get basis in input dict if exists
        basis = self.input_dict.get('basis', None)
        return basis
    
    def get_charge(self):
        # Get charge in input dict if exists
        charge = self.input_dict.get('charge', 0)
        return charge
    
    def get_spin(self):
        # Get spin in input dict if exists
        spin = self.input_dict.get('spin', 0)
        return spin
    
    def get_unit(self):
        # Get unit in input dict if exists
        # TODO: Should check the validity of the unit
        unit = self.input_dict.get('unit', 'Angstrom')
        return unit
    
    def get_restricted(self):
        # Get restricted in input dict if exists
        restricted = self.input_dict.get('restricted', True)
        return restricted
    
    def get_conv_tol(self):
        # Get conv_tol in input dict if exists
        conv_tol = self.input_dict.get('conv_tol', 1e-12)
        return conv_tol
    
    def get_conv_tol_grad(self):
        # Get conv_tol_grad in input dict if exists
        conv_tol_grad = self.input_dict.get('conv_tol_grad', 1e-8)
        return conv_tol_grad
    
    def get_direct_scf_tol(self):
        # Get direct_scf_tol in input dict if exists
        direct_scf_tol = self.input_dict.get('direct_scf_tol', 1e-13)
        return direct_scf_tol
    
    def get_init_guess(self):
        # Get init_guess in input dict if exists
        init_guess = self.input_dict.get('init_guess', 'minao')
        return init_guess
    
    def get_level_shift(self):
        # Get level_shift in input dict if exists
        level_shift = self.input_dict.get('level_shift', 0.0)
        return level_shift
    
    def get_max_cycle(self):
        # Get max_cycle in input dict if exists
        max_cycle = self.input_dict.get('max_cycle', 100)
        return max_cycle
    
    def get_max_memory(self):
        # Get max_memory in input dict if exists
        max_memory = self.input_dict.get('max_memory', 8000)
        return max_memory
    
    def get_xc(self):
        # Get xc in input dict if exists
        xc = self.input_dict.get('xc', None)
        return xc
    
    def get_nlc(self):
        # Get nlc in input dict if exists
        nlc = self.input_dict.get('nlc', None)
        return nlc
    
    def get_xc_grid(self):
        # Get xc_grid in input dict if exists
        xc_grid = self.input_dict.get('xc_grid', 3)
        return xc_grid
    
    def get_nlc_grid(self):
        # Get nlc_grid in input dict if exists
        nlc_grid = self.input_dict.get('nlc_grid', 3)
        return nlc_grid
    
    def get_small_rho_cutoff(self):
        # Get small_rho_cutoff in input dict if exists
        small_rho_cutoff = self.input_dict.get('small_rho_cutoff', 1e-7)
        return small_rho_cutoff
    
    def get_atomic_radii(self):
        # Get atomic_radii in input dict if exists
        atomic_radii = self.input_dict.get('atomic_radii', 'BRAGG')
        atomic_radii.upper()
        return atomic_radii
    
    def get_becke_scheme(self):
        # Get becke_scheme in input dict if exists
        becke_scheme = self.input_dict.get('becke_scheme', 'BECKE')
        becke_scheme.upper()
        return becke_scheme
    
    def get_prune(self):
        # Get prune in input dict if exists
        prune = self.input_dict.get('prune', 'NWCHEM')
        if isinstance(prune, str):
            prune = prune.upper()
        return prune
    
    def get_radi_method(self):
        # Get radi_method in input dict if exists
        radi_method = self.input_dict.get('radi_method', 'TREUTLER_AHLRICHS')
        radi_method.upper()
        return radi_method
    
    def get_radii_adjust(self):
        # Get radii_adjust in input dict if exists
        radii_adjust = self.input_dict.get('radii_adjust', 'TREUTLER')
        if isinstance(radii_adjust, str):
            radii_adjust.upper()
        return radii_adjust
    
    def get_algorithm(self):
        # Get algorithm in input dict if exists
        algorithm = self.input_dict.get('algorithm', 'DIIS')
        algorithm.upper()
        return algorithm
    
    def get_datapath(self):
        # Get datapath in input dict if exists
        datapath = self.input_dict.get('datapath', None)
        return datapath
    
    def get_getdict(self):
        # Get getdict in input dict if exists
        getdict = self.input_dict.get('getdict', False)
        return getdict
    
    def get_lin_depth(self):
        # Get lin_depth in input dict if exists
        lin_depth = self.input_dict.get('lin_depth', 1e-8)
        return lin_depth
    
    def get_prop(self):
        # Get prop in input dict if exists
        prop = self.input_dict.get('prop', False)
        return prop
    
    def get_scf_type(self):
        # Get scf_type in input dict if exists
        scf_type = self.input_dict.get('scf_type', None)
        if scf_type:
            scf_type.upper()
        return scf_type
    
    def get_stable(self):
        # Get stable in input dict if exists
        stable = self.input_dict.get('stable', False)
        return stable
    
    def get_stable_cyc(self):
        # Get stable_cyc in input dict if exists
        stable_cyc = self.input_dict.get('stable_cyc', 4)
        return stable_cyc
    
    def get_verbose(self):
        # Get verbose in input dict if exists
        verbose = self.input_dict.get('verbose', 4)
        return verbose
    
    def get_job_type(self):
        job_type = self.input_dict['job_type']
        return job_type
    
    def get_ts_status(self):
        is_ts = self.input_dict['is_ts']
        if isinstance(is_ts, str):
            if is_ts.upper() == 'TRUE':
                is_ts = True
            elif is_ts.upper() == 'FALSE':
                is_ts = False
            else:
                raise ValueError('Invalid is_ts.')
        return is_ts
    
    def read_molecule(self, path):

        charge = spin = 0
        with open(path, 'r') as myfile:
            output = myfile.read()
            output = output.lstrip()
            output = output.rstrip()
            output = output.split('\n')

        try:
            int(output[0])
        except ValueError:
            try:
                charge = int(output[0].split(' ')[0])
                spin = int(output[0].split(' ')[1]) - 1
            except ValueError:
                molecule = output
            else:
                molecule = '\n'.join(output[1:])
        else:
            if int(output[0]) == len(output) - 2:
                molecule = '\n'.join(output[2:])
                try:
                    charge = int(output[1].split(' ')[0])
                    spin = int(output[1].split(' ')[1])-1
                except ValueError:
                    pass
            else:
                print ("THIS IS NOT A VALID XYZ FILE")

        return (molecule, charge, spin)

    def convert_to_custom_xyz(self, atoms_and_coordinates):
        BOHR_TO_ANGSTROM = 0.52917721067 # Conversion factor from Bohr to Ångstrom
        xyz_data = []
        
        for atom, coordinates in atoms_and_coordinates:
            # Convert each coordinate from Bohr to Ångstrom
            converted_coordinates = [coord * BOHR_TO_ANGSTROM for coord in coordinates]

            atom_line = f"{atom:<2}      {converted_coordinates[0]: .8f}   {converted_coordinates[1]: .8f}   {converted_coordinates[2]: .8f}"
            xyz_data.append(atom_line)
        
        return '\n'.join(xyz_data)
    
    def run(self):
        
        mol = gto.Mole()
        
        # Set mol attributes
        try:
            gto.Mole(atom=self.molecule, charge=self.charge, spin =self.spin).build()
        except KeyError:
            # Probable not relevant
             (mol.atom, charge, spin) = self.read_molecule(self.datapath + self.molecule)
        else:
            mol.atom = self.molecule
        mol.basis = self.basis
        mol.charge = self.charge
        mol.spin = self.spin
        mol.unit = self.unit
        mol.verbose = self.verbose
        mol.build()
        
        # Check method for density functional
        DFT = False
        if self.method != 'HF':
            try:
                # https://pyscf.org/_modules/pyscf/dft/libxc.html
                dft.libxc.parse_xc(self.method)
            except (ValueError, KeyError):
                pass
            else:
                # Kohn-Sham (KS)
                self.xc = self.method
                self.method = 'KS'
                
        # Check if restricted
        if not self.restricted and self.method in ['HF', 'KS', 'DFT'] and self.scf_type is None:
            self.scf_type = 'R'
        else:
            self.scf_type = 'U'
            
        # Create HF/KS/DFT object
        if self.method in ['RHF', 'ROHF'] or (self.method == 'HF' and self.scf_type in ['R', 'RO']):
            mf = scf.RHF(mol)
            self.scf_type = 'R'
        elif self.method == 'UHF' or (self.method == 'HF' and self.scf_type == 'U'):
            mf = scf.UHF(mol)
            self.scf_type = 'U'
        elif self.method in ['RKS', 'ROKS', 'RDFT', 'RODFT'] or (self.method in ['KS', 'DFT'] and self.scf_type in ['R', 'RO']):
            mf = dft.RKS(mol)
            self.scf_type = 'R'
            DFT = True
        elif self.method in ['UKS', 'UDFT'] or (self.method in ['KS', 'DFT'] and self.scf_type == 'U'):
            mf = dft.UKS(mol)
            self.scf_type = 'U'
            DFT = True
        else:
            raise ValueError('Invalid method.')
        
        # Set HF attributes
        mf.conv_check = False
        mf.conv_tol = self.conv_tol
        mf.conv_tol_grad = self.conv_tol_grad
        mf.direct_scf_tol = self.direct_scf_tol
        mf.init_guess = self.init_guess
        mf.level_shift = self.level_shift
        mf.max_cycle = self.max_cycle
        mf.max_memory = self.max_memory
        mf.verbose = self.verbose
        
        # Set KS attributes
        if DFT:
            mf.xc = self.xc
            if mf.xc is None:
                raise ValueError('No XC functional specified.')
            mf.nlc = self.nlc
            
            if isinstance(self.xc_grid, int):
                mf.grids.level = self.xc_grid
            elif isinstance(self.xc_grid, tuple) or isinstance(self.xc_grid, dict):
                mf.grids.atom_grid = self.xc_grid
            else:
                raise ValueError('Invalid xc_grid.')
            
            if isinstance(self.nlc_grid, int):
                mf.nlcgrids.level = self.nlc_grid
            elif isinstance(self.nlc_grid, tuple) or isinstance(self.nlc_grid, dict):
                mf.nlcgrids.atom_grid = self.nlc_grid
            else:
                raise ValueError('Invalid nlc_grid.')
            
            if self.atomic_radii == 'BRAGG':
                mf.grids.radi_method = dft.radi.BRAGG_RADII
                mf.nlcgrids.radi_method = dft.radi.BRAGG_RADII
            elif self.atomic_radii == 'COVALENT':
                mf.grids.radi_method = dft.radi.COVALENT_RADII
                mf.nlcgrids.radi_method = dft.radi.COVALENT_RADII
            else:
                raise ValueError('Invalid atomic_radii.')
            
            if self.becke_scheme == 'BECKE':
                mf.grids.becke_scheme = dft.gen_grid.original_becke
                mf.nlcgrids.becke_scheme = dft.gen_grid.original_becke
            elif self.becke_scheme == 'STRATMANN':
                mf.grids.becke_scheme = dft.gen_grid.stratmann
                mf.nlcgrids.becke_scheme = dft.gen_grid.stratmann
            else:
                raise ValueError('Invalid becke_scheme.')
            
            if self.prune == 'NWCHEM':
                mf.grids.prune = dft.gen_grid.nwchem_prune
                mf.nlcgrids.prune = dft.gen_grid.nwchem_prune
            elif self.prune == 'SG1':
                mf.grids.prune = dft.gen_grid.sg1_prune
                mf.nlcgrids.prune = dft.gen_grid.sg1_prune
            elif self.prune == 'TREUTLER':
                mf.grids.prune = dft.gen_grid.treutler_prune
                mf.nlcgrids.prune = dft.gen_grid.treutler_prune
            elif self.prune == 'NONE' or self.prune is None:
                mf.grids.prune = None
                mf.nlcgrids.prune = None
            else:
                raise ValueError('Invalid prune.')
            
            if self.radi_method in ['TREUTLER_AHLRICHS', 'TREUTLER', 'AHLRICHS']:
                mf.grids.radi_method = dft.radi.treutler_ahlrichs
                mf.nlcgrids.radi_method = dft.radi.treutler_ahlrichs
            elif self.radi_method == 'DELLEY':
                mf.grids.radi_method = dft.radi.delley
                mf.nlcgrids.radi_method = dft.radi.delley
            elif self.radi_method in ['MURA_KNOWLES', 'MURA', 'KNOWLES']:
                mf.grids.radi_method = dft.radi.mura_knowles
                mf.nlcgrids.radi_method = dft.radi.mura_knowles
            elif self.radi_method in ['GAUSS_CHEBYSHEV', 'GAUSS', 'CHEBYSHEV']:
                mf.grids.radi_method = dft.radi.gauss_chebyshev
                mf.nlcgrids.radi_method = dft.radi.gauss_chebyshev
            else:
                raise ValueError('Invalid radi_method.')
            
            if self.radii_adjust == 'TREUTLER':
                mf.grids.radii_adjust = dft.radi.treutler_atomic_radii_adjust
                mf.nlcgrids.radii_adjust = dft.radi.treutler_atomic_radii_adjust
            elif self.radii_adjust == 'BECKE':
                mf.grids.radii_adjust = dft.radi.becke_atomic_radii_adjust
                mf.nlcgrids.radii_adjust = dft.radi.becke_atomic_radii_adjust
            elif self.radii_adjust == 'NONE' or self.radii_adjust is None:
                mf.grids.radii_adjust = None
                mf.nlcgrids.radii_adjust = None
            else:
                raise ValueError('Invalid radii_adjust.')
            
            mf.small_rho_cutoff = self.small_rho_cutoff
        
        # Select Optimizer
        if self.algorithm == 'DIIS':
            mf.diis = True
        elif self.algorithm == 'EDIIS':
            mf.diis = scf.diis.EDIIS()
        elif self.algorithm == 'ADIIS':
            mf.diis = scf.diis.ADIIS()
        elif self.algorithm == 'NEWTON':
            mf = mf.newton()
        else:
            raise ValueError('Invalid algorithm.')
        
        self.mf = mf
        # Running the Kernel
        
                # TODO: Check if TS
        if self.is_ts and self.job_type == 'opt':
            from pyscf.qsdopt.qsd_optimizer import QSD
            optimizer = QSD(self.mf, stationary_point='TS')
            

            # hess_update_freq: Frequency for numerical reevaluation of hessian. = 0 evaluates the numerical hessian in the first iteration and is updated with an BFGS rule, unless approaching a trap region, where it is reevaluated. (Default: 0)

            # numhess_method: Method for evaluating numerical hessian. Forward and central differences are available with “forward” and “central”, respectively. (Default: “forward”)

            # max_iter: Maximum number of optimization steps. (Default: 100)

            # step: Maximum norm between two optimization steps. (Default: 0.1)

            # hmin: Minimum distance between current geometry and stationary point of the quadratic form to consider convergence reached. (Default: 1e-6)

            # gthres: Gradient norm to consider convergence. (Default: 1e-5)

            self.mol_eq = optimizer.kernel(hess_update_freq=0, numhess_method='forward', max_iter=100, step=0.1, hmin=1e-6, gthres=1e-5)
            
        
        if self.job_type == 'opt' and not self.is_ts:
            from pyscf.geomopt.geometric_solver import optimize
            conv_params = { # These are the default settings
                                'convergence_energy': 1e-6,  # Eh
                                'convergence_grms': 3e-4,    # Eh/Bohr
                                'convergence_gmax': 4.5e-4,  # Eh/Bohr
                                'convergence_drms': 1.2e-3,  # Angstrom
                                'convergence_dmax': 1.8e-3,  # Angstrom
                            }
            self.mol_eq = optimize(self.mf, maxsteps=100, **conv_params).kernel() # TODO: Need to define maxsteps as a TRSH parameter
            print(self.mol_eq.atom)
            print('In Angstroms')
            print(self.convert_to_custom_xyz(self.mol_eq.atom))
            ### Save the optimized geometry to the a YAML file
            
            print('SCF energy of {0} is {1}.'.format(self.method, self.mf.e_tot))
            
        if self.job_type == 'freq':
            #npip install git+https://github.com/pyscf/properties
            from pyscf.prop.freq import rks, uks
            self.mol_eq = self.mf.run()
            if DFT:
                if self.scf_type == 'R':
                    w, modes = rks.Freq(self.mol_eq).kernel()
                elif self.scf_type == 'U':
                    w, modes = uks.Freq(self.mol_eq).kernel()
            
            print('Frequencies (cm-1):')
            print(w)
            print('Modes:')
            print(modes)
            
        
        # if self.job_type == 'scan':
        #     #https://github.com/pyscf/pyscf/blob/14d88828cd1f18f1e5358da1445355bde55322a1/examples/scf/30-scan_pes.py#L16
        #     # Requires manual development for the code. Not feasible at the moment.
        #     scan_results = []
        #     for value in np.arange(self.start, self.end, self.step_size):
        #         upd
        if self.job_type == 'sp':
            self.mf.kernel()
            print('The single-point DFT energy is:', energy)

In [26]:
pyscf_class = PYSCFScript_VB('/home/calvin/Code/ARC/arc/testing/test_PYSCFAdapter/calcs/Species/EtOH/opt_a205/input.yml')

In [27]:
pyscf_class.job_type


'opt'

In [28]:
pyscf_class.mf().unit

AttributeError: 'PYSCFScript_VB' object has no attribute 'mf'

In [29]:
pyscf_class.run()

System: uname_result(system='Linux', node='calvin-System-Product-Name', release='6.2.0-37-generic', version='#38~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov  2 18:01:13 UTC 2', machine='x86_64')  Threads 20
Python 3.10.13 | packaged by conda-forge | (main, Oct 26 2023, 18:07:37) [GCC 12.3.0]
numpy 1.26.0  scipy 1.11.3
Date: Thu Nov 23 15:32:12 2023
PySCF version 2.4.0
PySCF path  /home/calvin/mambaforge/envs/pyscf_env/lib/python3.10/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 9
[INPUT] num. electrons = 26
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = Angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 C     -0.937906740000   0.280664430000   0.105729420000 AA   -1.772386868984   0.530378905607   0.199799647109 Bohr   0.0
[INPUT]  2 C      0.356599060000  -0.449549970000   

geometric-optimize called with the following command line:
/home/calvin/mambaforge/envs/pyscf_env/lib/python3.10/site-packages/ipykernel_launcher.py --f=/home/calvin/.local/share/jupyter/runtime/kernel-v2-61254VeQFmWJ8V51.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **          


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C  -0.937907   0.280664   0.105729    0.000000  0.000000  0.000000
   C   0.356599  -0.449550   0.050202    0.000000  0.000000  0.000000
   O   0.366265  -1.593980  -0.380126    0.000000  0.000000  0.000000
   H  -1.689239  -0.333322   0.613292    0.000000  0.000000  0.000000
   H  -0.855320   1.239100   0.625780    0.000000  0.000000  0.000000
   H  -1.307049   0.460012  -0.909489    0.000000  0.000000  0.000000
   H   0.762810  -0.500366   1.064830    0.000000  0.000000  0.000000
   H   1.042871   0.121376  -0.582361    0.000000  0.000000  0.000000
   H   1.278200  -1.930310  -0.352035    0.000000  0.000000  0.000000
New geometry
   1 C     -0.937906740000   0.280664430000   0.105729420000 AA   -1.772386868984   0.530378905607   0.199799647109 Bohr

   2 C      0.356599060000  -0.449549970000   0.050201740000 AA    0.673874559677  -0.849526322606   0.0948

Step    0 : Gradient = 1.031e-01/2.334e-01 (rms/max) Energy = -155.0649840895
Hessian Eigenvalues: 2.30000e-02 2.30000e-02 5.00000e-02 ... 3.44045e-01 5.28926e-01 9.41969e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C  -0.938350   0.269964   0.099340   -0.000443 -0.010701 -0.006389
   C   0.416026  -0.398464   0.080343    0.059426  0.051086  0.030142
   O   0.337138  -1.647696  -0.406247   -0.029127 -0.053716 -0.026121
   H  -1.673897  -0.365923   0.603067    0.015342 -0.032601 -0.010224
   H  -0.894297   1.229997   0.613029   -0.038977 -0.009103 -0.012752
   H  -1.308070   0.432154  -0.918213   -0.001021 -0.027857 -0.008724
   H   0.802316  -0.378894   1.122467    0.039506  0.121472  0.057637
   H   1.086169   0.245402  -0.529116    0.043298  0.124026  0.053245
   H   1.190035  -2.093450  -0.428609   -0.088165 -0.163140 -0.076575
New geometry
   1 C     -0.938349619267   0.269963521994   0.099340495726 AA   -1.773223789504   0.510157120191   0.187726330002 Bohr

   2 C      0.416025532685  -0.398464349439   0.080343405930 AA    0.786174317601  -0.752988490843   0.1518

...


Geometry optimization cycle 9
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C  -0.951815   0.278678   0.098964   -0.001039  0.001043  0.000420
   C   0.420313  -0.359200   0.111949   -0.000825 -0.000395  0.000810
   O   0.310990  -1.657837  -0.471405   -0.000932  0.003677 -0.004441
   H  -1.660029  -0.323760   0.670186    0.000223  0.004960  0.005864
   H  -0.912839   1.277082   0.539263    0.001265  0.003704 -0.005832
   H  -1.324580   0.365413  -0.922938   -0.001817 -0.004966  0.000322
   H   0.792566  -0.432120   1.141788   -0.003090 -0.003817  0.001434
   H   1.128403   0.257691  -0.455811    0.001035  0.002615  0.006248
   H   1.181186  -2.069046  -0.481548   -0.001911  0.001619 -0.003542
New geometry
   1 C     -0.951814641630   0.278677621029   0.098964267826 AA   -1.798668994032   0.526624380791   0.187015362309 Bohr

   2 C      0.420313015294  -0.359200472756   0.111949334358 AA    0.794276485495  -0.678790517323   0.2115

Step    8 : Displace = 5.235e-03/7.278e-03 (rms/max) Trust = 3.000e-01 (=) Grad = 2.583e-04/4.984e-04 (rms/max) E (change) = -155.1096003124 (-5.475e-06) Quality = 1.211
Hessian Eigenvalues: 2.92609e-03 2.21768e-02 4.90599e-02 ... 3.47030e-01 4.84479e-01 5.34979e-01



Geometry optimization cycle 10
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C  -0.952471   0.279298   0.099162   -0.000656  0.000621  0.000198
   C   0.420077  -0.359011   0.111863   -0.000236  0.000189 -0.000086
   O   0.310012  -1.656723  -0.471970   -0.000978  0.001114 -0.000564
   H  -1.659966  -0.323252   0.670959    0.000063  0.000508  0.000773
   H  -0.912983   1.277703   0.539362   -0.000144  0.000622  0.000099
   H  -1.324780   0.365912  -0.922806   -0.000200  0.000499  0.000132
   H   0.792398  -0.431846   1.141692   -0.000168  0.000274 -0.000096
   H   1.128283   0.258441  -0.455181   -0.000120  0.000750  0.000630
   H   1.179483  -2.069483  -0.480864   -0.001703 -0.000437  0.000684
New geometry
   1 C     -0.952471068442   0.279298322465   0.099162056597 AA   -1.799909460927   0.527797336510   0.187389128916 Bohr

   2 C      0.420077001355  -0.359011249945   0.111863300616 AA    0.793830483790  -0.678432938035   0.211

Step    9 : Displace = 6.794e-04/1.127e-03 (rms/max) Trust = 3.000e-01 (=) Grad = 4.713e-05/7.609e-05 (rms/max) E (change) = -155.1096010816 (-7.692e-07) Quality = 1.103
Hessian Eigenvalues: 2.92609e-03 2.21768e-02 4.90599e-02 ... 3.47030e-01 4.84479e-01 5.34979e-01
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 221.518 seconds


[('C', [-1.7999094609272286, 0.5277973365100489, 0.18738912891640458]), ('C', [0.7938304837896802, -0.6784329380345204, 0.21139100155442303]), ('O', [0.5858372799846862, -3.130752670900494, -0.8918934735394523]), ('H', [-3.13688145707253, -0.6108575935804842, 1.2679281282361141]), ('H', [-1.7252882102893343, 2.4145094479136175, 1.0192466015971782]), ('H', [-2.503471225306489, 0.6914731252978847, -1.743850950414648]), ('H', [1.4974145866889246, -0.8160702330472801, 2.157485582222608]), ('H', [2.1321453069048526, 0.48838202703370726, -0.8601672784666464]), ('H', [2.2288989493543028, -3.910756426766914, -0.9087019480428279])]
In Angstroms
C       -0.95215210    0.27920479    0.09912885
C        0.41993633   -0.35889102    0.11182584
O        0.30990792   -1.65616816   -0.47181165
H       -1.65941029   -0.32314367    0.67073398
H       -0.91267746    1.27727550    0.53918145
H       -1.32433628    0.36578928   -0.92249715
H        0.79213232   -0.43170115    1.14130987
H        1.12790487 

In [19]:
pyscf_class.mol_eq.atom

[('C', [-1.7999094609202635, 0.5277973365129489, 0.18738912891956164]),
 ('C', [0.793830483797095, -0.6784329380304922, 0.21139100155990073]),
 ('O', [0.5858372799935844, -3.1307526708978766, -0.8918934735312511]),
 ('H', [-3.136881457065631, -0.6108575935768236, 1.2679281282400343]),
 ('H', [-1.7252882102835332, 2.4145094479175055, 1.0192466015982453]),
 ('H', [-2.5034712252988904, 0.6914731252982219, -1.7438509504119493]),
 ('H', [1.497414586695588, -0.81607023304081, 2.1574855822285284]),
 ('H', [2.1321453069122622, 0.4883820270368722, -0.8601672784620991]),
 ('H', [2.2288989493637055, -3.910756426763262, -0.9087019480339279])]

In [61]:
pyscf_class.mol_eq.diis



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmp27oyta47
max_memory 4000 MB (current use 313 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
init E= -154.358637795895
  HOMO = -0.335866613147092  LUMO = 0.0874179608128103
cycle= 1 E= -154.084014086742  delta_E= 0.275  |g|= 0.517  |ddm|= 1.85
  HOMO = -0.394871760220418  LUMO = 0.126143019763058
cycle= 2 E= -154.130388776101  delta_E= -0.0464  |g|= 0.294  |ddm|= 0.45
  HOMO = -0.446553393528937  LUMO = 0.132323696265205
cycle= 3 E= -154.144917184244  delta_E= -0.0145  |g|= 0.0504  |ddm|= 0.177
  HOMO = -0.438784534632169  LUMO = 0.135957813874833
cycle= 4 E= -154.145624872644  delta_E= -0.000708  |g|= 0.00959  |ddm|= 0.0426

True

In [63]:
def convert_to_xyz(atoms_and_coordinates, comment="Molecule in XYZ format"):
    num_atoms = len(atoms_and_coordinates)
    xyz_data = [f"{num_atoms}\n{comment}"]
    
    for atom, coordinates in atoms_and_coordinates:
        atom_line = f"{atom:<2} {coordinates[0]:.8f} {coordinates[1]:.8f} {coordinates[2]:.8f}"
        xyz_data.append(atom_line)
    
    return "\n".join(xyz_data)

In [64]:
convert_to_xyz(pyscf_class.mol_eq.atom)

'9\nMolecule in XYZ format\nC  -1.79990946 0.52779734 0.18738913\nC  0.79383048 -0.67843294 0.21139100\nO  0.58583728 -3.13075267 -0.89189347\nH  -3.13688146 -0.61085759 1.26792813\nH  -1.72528821 2.41450945 1.01924660\nH  -2.50347122 0.69147313 -1.74385095\nH  1.49741459 -0.81607023 2.15748558\nH  2.13214531 0.48838203 -0.86016728\nH  2.22889895 -3.91075643 -0.90870195'

In [72]:
def convert_to_custom_xyz(atoms_and_coordinates):
    xyz_data = []
    
    for atom, coordinates in atoms_and_coordinates:
        atom_line = f"{atom:<2}      {coordinates[0]: .8f}   {coordinates[1]: .8f}   {coordinates[2]: .8f}"
        xyz_data.append(atom_line)
    
    return '\n'.join(xyz_data)

In [73]:
print(convert_to_custom_xyz(pyscf_class.mol_eq.atom))

C       -1.79990946    0.52779734    0.18738913
C        0.79383048   -0.67843294    0.21139100
O        0.58583728   -3.13075267   -0.89189347
H       -3.13688146   -0.61085759    1.26792813
H       -1.72528821    2.41450945    1.01924660
H       -2.50347122    0.69147313   -1.74385095
H        1.49741459   -0.81607023    2.15748558
H        2.13214531    0.48838203   -0.86016728
H        2.22889895   -3.91075643   -0.90870195


In [70]:
print(pyscf_class.molecule)

C      -0.93790674    0.28066443    0.10572942
C       0.35659906   -0.44954997    0.05020174
O       0.36626530   -1.59397979   -0.38012632
H      -1.68923915   -0.33332195    0.61329151
H      -0.85532021    1.23909997    0.62578027
H      -1.30704889    0.46001151   -0.90948878
H       0.76281007   -0.50036590    1.06483009
H       1.04287051    0.12137561   -0.58236096
H       1.27820018   -1.93031032   -0.35203473


In [69]:
pyscf_class.input_dict

{'basis': 'def2-tzvp',
 'charge': 0,
 'is_ts': 'False',
 'job_type': 'opt',
 'restricted': 'False',
 'spin': 0,
 'xc_func': 'b3lyp',
 'xyz': 'C      -0.93790674    0.28066443    0.10572942\nC       0.35659906   -0.44954997    0.05020174\nO       0.36626530   -1.59397979   -0.38012632\nH      -1.68923915   -0.33332195    0.61329151\nH      -0.85532021    1.23909997    0.62578027\nH      -1.30704889    0.46001151   -0.90948878\nH       0.76281007   -0.50036590    1.06483009\nH       1.04287051    0.12137561   -0.58236096\nH       1.27820018   -1.93031032   -0.35203473'}

In [ ]:
#P guess=mix wb97xd/def2tzvp freq IOp(7/33=1)  integral=(grid=ultrafine, Acc2E=12)  IOp(2/9=2000)   scf=(tight, direct,xqc)

rxn_3_OO

0 1
O       0.59489400   -0.38529500    0.26586700
O      -0.59175400    0.40282200    0.24616500
H       1.18646100    0.15390500   -0.26887000
H      -1.18960100   -0.17143300   -0.24316200



In [121]:
freq_dict = {'job_type': 'freq',
             'basis': 'def2-TZVP',
             'xc_func': 'wb97xd',
             'is_ts': False,
             'charge': 0,
             'spin': 0,
             'verbose': 0,
             'xyz': 'O       0.59489400   -0.38529500    0.26586700\nO      -0.59175400    0.40282200    0.24616500\nH       1.18646100    0.15390500   -0.26887000\nH      -1.18960100   -0.17143300   -0.24316200',
}
             

In [122]:
pyscf_freq = PYSCFScript_VB(freq_dict)

In [123]:
pyscf_freq.input_dict

{'job_type': 'freq',
 'basis': 'def2-TZVP',
 'xc_func': 'wb97xd',
 'is_ts': False,
 'charge': 0,
 'spin': 0,
 'verbose': 0,
 'xyz': 'O       0.59489400   -0.38529500    0.26586700\nO      -0.59175400    0.40282200    0.24616500\nH       1.18646100    0.15390500   -0.26887000\nH      -1.18960100   -0.17143300   -0.24316200'}

In [124]:
pyscf_freq.run()

Frequencies (cm-1):
[ -69.17402564  -43.06833548  -36.16940695  -14.10742842   48.42207691
  167.57213865  424.92332344 1016.95977536 1364.77346519 1483.83055979
 3834.03013272 3834.65834819]
Modes:
[[[-5.49868780e-01  9.64767744e-02  4.36356704e-01]
  [-5.07247738e-01  1.38575332e-01 -4.35165718e-01]
  [-9.15799725e-02 -7.24867039e-02  6.34330708e-02]
  [-9.63266776e-02 -4.00549632e-02 -5.92333883e-02]]

 [[ 2.94217778e-01  5.23586183e-01  3.71586392e-01]
  [ 3.40342000e-01  5.80324227e-01 -1.48107133e-01]
  [ 5.84231421e-02 -5.94321829e-04 -5.69386471e-02]
  [ 5.37906248e-02  4.12652632e-02  1.24179198e-01]]

 [[ 3.51140707e-01  2.07532650e-01 -2.32379299e-01]
  [-1.50376983e-02 -3.55996224e-01 -7.39113239e-01]
  [ 6.51832461e-02  9.57874449e-02 -3.99100355e-02]
  [ 1.06284765e-01 -1.69682164e-01 -2.26444368e-01]]

 [[ 7.47929402e-02 -6.14677637e-01  4.93095237e-01]
  [ 4.28284599e-01 -1.01892746e-01 -2.42333074e-01]
  [ 1.33954505e-01 -2.49744868e-01  1.55441335e-01]
  [ 9.63471000e